In [1]:
import ee
import geemap
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import folium

DATA_PATH = "../data/"

try:
    # Initialize the library.
    ee.Initialize(project='')
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def image_date(image):
    # Get the image's 'system:time_start' property, which represents the image acquisition time
    time_start = ee.Date(image.get('system:time_start'))
    # Format the date to a string and print it
    return time_start.format('YYYY-MM-dd').getInfo()
    
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

Google Earth Engine has initialized successfully!


In [2]:
### dataset loading
coordinates = [149.35704470513303,-34.98, 149.49550371751621,-35.20614998677668]
region = ee.Geometry.Rectangle(coordinates)

dataset = ( 
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterDate('1984-03-16', '2012-05-01')
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', 20)
)

dataset = dataset.map(apply_scale_factors)

listOfImages = dataset.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))
collection_list = dataset.toList(dataset.size())

img_id = 1
image = ee.Image(collection_list.get(img_id))
capture_date = image_date(image)

#95 looks empty but shows water inside

#messy but kept: 48,54,57,58,61,64,72,84,85,124

#shitlist: 9,20,23,27,29,32,35,36,55,59,60,71,74,78,108,116,126,136,

Number of images in the collection:  217


In [3]:
for i in range(0,142):
    if i in (9,20,23,27,29,32,35,36,55,59,60,71,74,78,108,116,126,136):
        continue
    else:
        img_id = i
        image = ee.Image(collection_list.get(img_id))
        capture_date = image_date(image)

        blue = image.select('SR_B1')
        green = image.select('SR_B2')
        swir1 = image.select('SR_B5')
        swir2 = image.select('SR_B7')
        nir = image.select('SR_B4')

        awei_sh = blue.add(green.multiply(2.5)).subtract((nir.add(swir1)).multiply(1.5)).subtract(swir2.multiply(0.25))

        filename_rgb = DATA_PATH + '/raw/george_' + str(img_id) + '_RGB_' + capture_date + '.tif'
        filename_awei = DATA_PATH + '/processed/george_' + str(img_id) + '_AWEI_' + capture_date + '.tif'

        geemap.ee_export_image(image, filename=filename_rgb, scale=30, region=region, file_per_band=False)
        geemap.ee_export_image(awei_sh, filename=filename_awei, scale=30, region=region, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\raw\george_0_RGB_1987-09-18.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\processed\george_0_AWEI_1987-09-18.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\raw\george_1_RGB_1987-11-21.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\processed\george_1_AWEI_1987-11-21.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\raw\george_2_RGB_1988-01-08.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Analysis\data\processed\george_2_AWEI_1988-01-08.tif
Generating URL ...
Please wait ...
Data downloaded to d:\Master-Of-Data-Science\2A\DS\project\Lakes-Ana

In [ ]:
img_id = 143
image = ee.Image(collection_list.get(img_id))
capture_date = image_date(image)
print(capture_date)

vis_RGB = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}

blue = image.select('SR_B1')
green = image.select('SR_B2')
swir1 = image.select('SR_B5')
swir2 = image.select('SR_B7')
nir = image.select('SR_B4')

awei_sh = blue.add(green.multiply(2.5)).subtract((nir.add(swir1)).multiply(1.5)).subtract(swir2.multiply(0.25))

filename_rgb = r'\Lakes-Analysis-main\data\raw\george_' + str(img_id) + '_RGB_' + capture_date + '.tif'
filename_awei = r'\Lakes-Analysis-main\data\processed\george_' + str(img_id) + '_AWEI_' + capture_date + '.tif'

#geemap.ee_export_image(image, filename=filename_rgb, scale=30, region=region, file_per_band=False)
#geemap.ee_export_image(awei_sh, filename=filename_awei, scale=10, region=region, file_per_band=False)

m_rgb = geemap.Map()
m_rgb.set_center(149.42, -35.08,11)
m_rgb.add_layer(image.clip(region), vis_RGB, 'RGB')
m_rgb

In [ ]:

vis_AWEI = {
  'min': 0,
  'max': 1,
  'palette': ['white', 'blue']
}

m_awei = geemap.Map()
m_awei.set_center(149.42, -35.08,10)
m_awei.add_layer(awei_sh.clip(region), vis_AWEI, 'AWEI (shadows)')
m_awei